# Operating Systems & Concurrency: system calls; processes
_COSC 208, Introduction to Computer Systems, 2024-04-08_

## Announcements
* Project 2 revisions due Friday at 11pm
* Quiz 5 rescheduled for Friday, April 19
* Office hours
    * Today – canceled
    * Tomorrow – 8:15am-9:45am and 12:45pm-1:15pm

## Outline
* Limited Direct Execution (LDE)
* System calls (syscalls)
* Process abstraction

## No Warm-up – Happy Solar Eclipse!

## Limited Direct Execution (LDE)

* CPU has two modes of execution: user mode & kernel mode
* _When does a CPU run in user mode?_ — when executing application code
* _When does a CPU run in kernel mode?_ — when executing OS code
* Allowable operations in user mode are restricted
    * Applications can...
        * Perform arithmetic/logic operations
        * Load/store values in its stack/heap
    * Applications must ask the OS to...
        * Start/terminate applications
        * Create/delete files
        * Display output on screen
        * Read input from user
    * Must transfer control to the OS to perform these operations — How?

## System calls (syscalls)

* Invoked via a special assembly instruction: trap (generic) or `svc` (on ARM)

* Example program

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int name() {
/* 4*/      int pid = getpid();
/* 5*/      return pid;
/* 6*/  }
/* 7*/  int main() {
/* 8*/      int p = name();
/* 9*/      printf("I am a process. My name is %d.\n", p);
/*10*/  }

I am a process. My name is 1335.


* Assembly code

```
00000000004006ac <name>:
    4006ac:    d10083ff     sub    sp, sp, #0x20
    4006b0:    f9000bfe     str    x30, [sp, #16]
    4006b4:    94007713     bl    41e300 <__getpid>
    4006b8:    b9000fe0     str    w0, [sp, #12]
    4006bc:    b9400fe0     ldr    w0, [sp, #12]
    4006c0:    f9400bfe     ldr    x30, [sp, #16]
    4006c4:    910083ff     add    sp, sp, #0x20
    4006c8:    d65f03c0     ret
000000000041e300 <__getpid>:
    41e300:    d28015c8     mov    x8, #0xac
    41e304:    d4000001     svc    #0x0
    41e308:    d65f03c0     ret
```

* Functions in the C standard library that involve a privileged operation (e.g., `printf`) put the system call number in a register and invoke a trap instruction — programmer doesn’t have to worry about these details; they can just call the appropriate function in the C standard library
* When `svc` is executed
    1. CPU switches to kernel mode
    2. CPU saves registers to the kernel stack — kernel stack is at a fixed location in memory
        * _Why do we need to save the registers?_ — so we can return to `user` when `__getuid` is done
    3. CPU uses system call number to index into table of trap handlers 
        * At boot, initialize table of trap handlers with pointers into OS code for handling each type of syscall
    4. Branch and link to trap handler code
    5. CPU restores registers from the kernel stack
    6. CPU switches to user mode
    7. Resume execution after `svc` 
* _What should we do if an application tries to perform a privileged operation without making a system call?_
    * Let the code keep running — code may assume privileged operation was successful
    * Terminate the application

<div style="page-break-after:always;"></div>

## Process abstraction

* Process — running program and its corresponding machine state (code, memory, and register values)
    * Program is static code and static data; process is dynamic instance of code and data
    * Cooking analogy
        * Recipe == program
        * Following a recipe == process
        * Ingredients == program inputs
        * Prepared food == program outputs
        * Cabinets, fridge, etc. == persistent storage
        * Prep area (e.g., counter) == registers & main memory
        * Contents and status of the prep area; current step of recipe == machine state
        * Chef == CPU
    * Can have multiple processes all running different instances of the same program
        * Cooking analogy — chef may be making multiple batches of the same recipe
* OS is responsible for...
    * Creating processes — when a user or another application requests it
    * Scheduling processes — i.e., deciding when/which process should be allowed to use the CPU
    * Switching processes — i.e., saving the machine state of one process and restoring the machine state of another process; called context switching
    * Cleaning-up processes — i.e., releasing any resources they are using when the process is done
    * Interacting with I/O devices (e.g., disks, NICs, graphics card) on behalf of processes

* Q1: _Consider building a Lego kit as an analogy for operating systems' process abstraction. Match each component of the analogy with the corresponding component of a real computer system._

* Analogy
    1. Cabinet/drawers for storing Legos
    2. Lego bricks
    3. Building area (e.g., tabletop)
    4. Instruction booklet
    5. Following the assembly instructions
    6. Current step for the instruction booklet
    7. Completed kit
    8. You
* Real system
    * CPU
    * persistent storage
    * process
    * program
    * program counter
    * program inputs
    * program outputs
    * registers and main memory

* Instruction booklet == program
* Following the assembly instructions == process
* Lego bricks == program inputs
* Completed kit == program outputs
* Cabinet/drawers for storing Legos == persistent storage
* Building area (e.g., tabletop) == registers and main memory
* Current step for the instruction booklet == program counter
* You == CPU